In [1]:
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
import os
import gc
import joblib
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics, preprocessing
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras import utils

In [2]:
def auc(y_true, y_pred):
    def fallback_auc(y_true, y_pred):
        try:
            return metrics.roc_auc_score(y_true, y_pred)
        except:
            return 0.5
    return tf.py_function(fallback_auc, (y_true, y_pred), tf.double)

In [3]:
def create_model(data, catcols):    
    inputs = []
    outputs = []
    for c in catcols:
        num_unique_values = int(data[c].nunique())
        embed_dim = int(min(np.ceil((num_unique_values)/2), 50))
        inp = layers.Input(shape=(1,))
        out = layers.Embedding(num_unique_values + 1, embed_dim, name=c)(inp)
        out = layers.SpatialDropout1D(0.3)(out)
        out = layers.Reshape(target_shape=(embed_dim, ))(out)
        inputs.append(inp)
        outputs.append(out)
    
    x = layers.Concatenate()(outputs)
    x = layers.BatchNormalization()(x)
    
    x = layers.Dense(300, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Dense(300, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)
    
    y = layers.Dense(2, activation="softmax")(x)

    model = Model(inputs=inputs, outputs=y)
    return model

In [4]:
train = pd.read_csv("input/train.csv")
test = pd.read_csv("input/test.csv")
sample = pd.read_csv("input/sample_submission.csv")

In [5]:
test["target"] = -1
data = pd.concat([train, test]).reset_index(drop=True)

In [6]:
features = [x for x in train.columns if x not in ["id", "target"]]

In [7]:
for feat in features:
    lbl_enc = preprocessing.LabelEncoder()
    data[feat] = lbl_enc.fit_transform(data[feat].values)

In [8]:
train = data[data.target != -1].reset_index(drop=True)
test = data[data.target == -1].reset_index(drop=True)
test_data = [test.loc[:, features].values[:, k] for k in range(test.loc[:, features].values.shape[1])]

In [9]:
np.array(test_data).shape

(23, 200000)

In [10]:
oof_preds = np.zeros((len(train)))
test_preds = np.zeros((len(test)))
cvs = []
skf = StratifiedKFold(n_splits=20)
for train_index, test_index in tqdm(skf.split(train, train.target.values)):
    X_train, X_test = train.iloc[train_index, :], train.iloc[test_index, :]
    X_train = X_train.reset_index(drop=True)
    X_test = X_test.reset_index(drop=True)
    y_train, y_test = X_train.target.values, X_test.target.values
    model = create_model(data, features)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[auc])
    X_train = [X_train.loc[:, features].values[:, k] for k in range(X_train.loc[:, features].values.shape[1])]
    X_test = [X_test.loc[:, features].values[:, k] for k in range(X_test.loc[:, features].values.shape[1])]
    
    es = callbacks.EarlyStopping(monitor='val_auc', min_delta=0.001, patience=5,
                                 verbose=1, mode='max', baseline=None, restore_best_weights=True)

    rlr = callbacks.ReduceLROnPlateau(monitor='val_auc', factor=0.5,
                                      patience=3, min_lr=1e-6, mode='max', verbose=1)
    
    model.fit(X_train,
              utils.to_categorical(y_train),
              validation_data=(X_test, utils.to_categorical(y_test)),
              verbose=1,
              batch_size=1024,
              callbacks=[es, rlr],
              epochs=100
             )
    valid_fold_preds = model.predict(X_test)[:, 1]
    test_fold_preds = model.predict(test_data)[:, 1]
    oof_preds[test_index] = valid_fold_preds.ravel()
    test_preds += test_fold_preds.ravel()
    cvs.append(metrics.roc_auc_score(y_test, valid_fold_preds))
    print(cvs[-1])
    K.clear_session()

0it [00:00, ?it/s]WARNING: Logging before flag parsing goes to stderr.
W0903 19:15:38.133915 139974362552128 deprecation.py:506] From /home/trent/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0903 19:15:38.925711 139974362552128 deprecation.py:506] From /home/trent/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0903 19:15:39.935714 139974362552128 deprecation.py:323] From /home/trent/anaconda3/lib/python3.7/site-packag

Train on 284999 samples, validate on 15001 samples
Epoch 1/100
284999/284999 [==============================] - 9s 33us/sample - loss: 0.6036 - auc: 0.6858 - val_loss: 0.5719 - val_auc: 0.7830
Epoch 2/100
284999/284999 [==============================] - 8s 27us/sample - loss: 0.5075 - auc: 0.7768 - val_loss: 0.4995 - val_auc: 0.7980
Epoch 3/100
284999/284999 [==============================] - 8s 27us/sample - loss: 0.4887 - auc: 0.7976 - val_loss: 0.4876 - val_auc: 0.7991
Epoch 4/100
284999/284999 [==============================] - 8s 27us/sample - loss: 0.4768 - auc: 0.8096 - val_loss: 0.4916 - val_auc: 0.7958
Epoch 5/100
284999/284999 [==============================] - 8s 26us/sample - loss: 0.4673 - auc: 0.8187 - val_loss: 0.4954 - val_auc: 0.7918
Epoch 6/100
284672/284999 [============================>.] - ETA: 0s - loss: 0.4560 - auc: 0.8284
Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
284999/284999 [==============================] - 8s 27us/samp

1it [01:33, 93.06s/it]

0.7993166085497173
Train on 284999 samples, validate on 15001 samples
Epoch 1/100
284999/284999 [==============================] - 9s 31us/sample - loss: 0.6009 - auc: 0.6851 - val_loss: 0.5676 - val_auc: 0.7722
Epoch 2/100
284999/284999 [==============================] - 7s 26us/sample - loss: 0.5070 - auc: 0.7772 - val_loss: 0.5054 - val_auc: 0.7862
Epoch 3/100
284999/284999 [==============================] - 7s 25us/sample - loss: 0.4881 - auc: 0.7984 - val_loss: 0.4989 - val_auc: 0.7851
Epoch 4/100
284999/284999 [==============================] - 7s 26us/sample - loss: 0.4766 - auc: 0.8101 - val_loss: 0.5046 - val_auc: 0.7805
Epoch 5/100
283648/284999 [============================>.] - ETA: 0s - loss: 0.4668 - auc: 0.8189
Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
284999/284999 [==============================] - 7s 26us/sample - loss: 0.4670 - auc: 0.8187 - val_loss: 0.5074 - val_auc: 0.7795
Epoch 6/100
284999/284999 [===========================

2it [02:53, 89.41s/it]

...

0.7975670098448115
Train on 285001 samples, validate on 14999 samples
Epoch 1/100
285001/285001 [==============================] - 9s 31us/sample - loss: 0.5991 - auc: 0.6856 - val_loss: 0.5692 - val_auc: 0.7827
Epoch 2/100
285001/285001 [==============================] - 7s 25us/sample - loss: 0.5066 - auc: 0.7778 - val_loss: 0.4954 - val_auc: 0.7994
Epoch 3/100
285001/285001 [==============================] - 7s 25us/sample - loss: 0.4878 - auc: 0.7986 - val_loss: 0.4874 - val_auc: 0.7989
Epoch 4/100
285001/285001 [==============================] - 7s 25us/sample - loss: 0.4769 - auc: 0.8099 - val_loss: 0.4879 - val_auc: 0.7979
Epoch 5/100
283648/285001 [============================>.] - ETA: 0s - loss: 0.4678 - auc: 0.8183
Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
285001/285001 [==============================] - 7s 25us/sample - loss: 0.4678 - auc: 0.8182 - val_loss: 0.4969 - val_auc: 0.7900
Epoch 6/100
285001/285001 [===========================

19it [26:42, 83.85s/it]

0.7994980202098294
Train on 285001 samples, validate on 14999 samples
Epoch 1/100
285001/285001 [==============================] - 9s 31us/sample - loss: 0.6014 - auc: 0.6865 - val_loss: 0.5660 - val_auc: 0.7766
Epoch 2/100
285001/285001 [==============================] - 7s 26us/sample - loss: 0.5082 - auc: 0.7761 - val_loss: 0.4999 - val_auc: 0.7939
Epoch 3/100
285001/285001 [==============================] - 7s 26us/sample - loss: 0.4883 - auc: 0.7980 - val_loss: 0.4903 - val_auc: 0.7950
Epoch 4/100
285001/285001 [==============================] - 7s 26us/sample - loss: 0.4777 - auc: 0.8087 - val_loss: 0.4916 - val_auc: 0.7935
Epoch 5/100
285001/285001 [==============================] - 7s 26us/sample - loss: 0.4672 - auc: 0.8189 - val_loss: 0.4962 - val_auc: 0.7898
Epoch 6/100
284672/285001 [============================>.] - ETA: 0s - loss: 0.4562 - auc: 0.8284
Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
285001/285001 [===========================

20it [28:11, 85.39s/it]

0.7949802879340031


In [11]:
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
def get_tuned_xgb(x,y): 
    # A parameter grid for XGBoost
    params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5],
        'n_estimators' : [50, 100, 200, 300, 500, 800, 1000]
        }
    
    xgb = XGBClassifier(objective='binary:logistic',n_jobs=8,)

    folds = 5
    param_comb = 15
    skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1003)
    random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='roc_auc', cv=skf.split(x,y), verbose=2, random_state=1003 )

    random_search.fit(x,y)
    print(random_search.best_score_)
    return XGBClassifier(**random_search.best_params_)

In [12]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
def get_tuned_lgbm(x,y): 
    # A parameter grid for XGBoost
    params = {'num_leaves': sp_randint(6, 50), 
             'min_child_samples': sp_randint(100, 500), 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': sp_uniform(loc=0.2, scale=0.8), 
             'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}
    
    lg = LGBMClassifier(objective='binary')

    folds = 5
    param_comb = 15
    skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1002)
    random_search = RandomizedSearchCV(lg, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=8, cv=skf.split(x,y), verbose=1, random_state=1002 )

    random_search.fit(x,y)
    print(random_search.best_score_)
    return LGBMClassifier(**random_search.best_params_)

In [13]:
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
def get_tuned_cat(x,y): 
    # A parameter grid for XGBoost
    params = {'depth':[3,1,2,6,4,5,7,8,9,10],
          'iterations':[2,50,100,500,1000],
          'learning_rate':[0.03,0.001,0.01,0.1,0.2,0.3], 
          'l2_leaf_reg':[3,1,5,10,100],
          'border_count':[32,5,10,20,50,100,200],
          'thread_count':[4],
              'loss_function': ['Logloss', 'CrossEntropy']}
    
    cat = CatBoostClassifier(eval_metric = 'AUC')

    folds = 5
    param_comb = 15
    skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)
    random_search = RandomizedSearchCV(cat, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=8, cv=skf.split(x,y), verbose=0, random_state=1001 )

    random_search.fit(x,y)
    print(random_search.best_score_)
    return CatBoostClassifier(**random_search.best_params_)

In [14]:
tcat = get_tuned_cat(train, train.target.values)
tlg = get_tuned_lgbm(train, train.target.values)
txgb = get_tuned_xgb(train, train.target.values)

0:	total: 103ms	remaining: 51.3s
1:	total: 147ms	remaining: 36.6s
2:	total: 200ms	remaining: 33.2s
3:	total: 252ms	remaining: 31.2s
4:	total: 300ms	remaining: 29.7s
5:	total: 344ms	remaining: 28.3s
6:	total: 391ms	remaining: 27.5s
7:	total: 433ms	remaining: 26.6s
8:	total: 475ms	remaining: 25.9s
9:	total: 525ms	remaining: 25.7s
10:	total: 568ms	remaining: 25.2s
11:	total: 610ms	remaining: 24.8s
12:	total: 657ms	remaining: 24.6s
13:	total: 699ms	remaining: 24.3s
14:	total: 748ms	remaining: 24.2s
15:	total: 794ms	remaining: 24s
16:	total: 834ms	remaining: 23.7s
17:	total: 881ms	remaining: 23.6s
18:	total: 932ms	remaining: 23.6s
19:	total: 981ms	remaining: 23.6s
20:	total: 1.03s	remaining: 23.5s
21:	total: 1.07s	remaining: 23.3s
22:	total: 1.12s	remaining: 23.3s
23:	total: 1.17s	remaining: 23.3s
24:	total: 1.22s	remaining: 23.1s
25:	total: 1.27s	remaining: 23.1s
26:	total: 1.31s	remaining: 22.9s
27:	total: 1.35s	remaining: 22.8s
28:	total: 1.39s	remaining: 22.6s
29:	total: 1.43s	remaining

242:	total: 10.3s	remaining: 10.9s
243:	total: 10.4s	remaining: 10.9s
244:	total: 10.4s	remaining: 10.8s
245:	total: 10.4s	remaining: 10.8s
246:	total: 10.5s	remaining: 10.7s
247:	total: 10.5s	remaining: 10.7s
248:	total: 10.6s	remaining: 10.6s
249:	total: 10.6s	remaining: 10.6s
250:	total: 10.6s	remaining: 10.6s
251:	total: 10.7s	remaining: 10.5s
252:	total: 10.7s	remaining: 10.5s
253:	total: 10.8s	remaining: 10.4s
254:	total: 10.8s	remaining: 10.4s
255:	total: 10.8s	remaining: 10.3s
256:	total: 10.9s	remaining: 10.3s
257:	total: 10.9s	remaining: 10.2s
258:	total: 11s	remaining: 10.2s
259:	total: 11s	remaining: 10.2s
260:	total: 11s	remaining: 10.1s
261:	total: 11.1s	remaining: 10.1s
262:	total: 11.1s	remaining: 10s
263:	total: 11.2s	remaining: 9.98s
264:	total: 11.2s	remaining: 9.94s
265:	total: 11.2s	remaining: 9.9s
266:	total: 11.3s	remaining: 9.85s
267:	total: 11.3s	remaining: 9.81s
268:	total: 11.4s	remaining: 9.77s
269:	total: 11.4s	remaining: 9.72s
270:	total: 11.4s	remaining: 

479:	total: 20.2s	remaining: 841ms
480:	total: 20.2s	remaining: 799ms
481:	total: 20.3s	remaining: 757ms
482:	total: 20.3s	remaining: 715ms
483:	total: 20.4s	remaining: 673ms
484:	total: 20.4s	remaining: 631ms
485:	total: 20.4s	remaining: 589ms
486:	total: 20.5s	remaining: 547ms
487:	total: 20.5s	remaining: 505ms
488:	total: 20.6s	remaining: 463ms
489:	total: 20.6s	remaining: 421ms
490:	total: 20.7s	remaining: 379ms
491:	total: 20.7s	remaining: 337ms
492:	total: 20.7s	remaining: 295ms
493:	total: 20.8s	remaining: 252ms
494:	total: 20.8s	remaining: 210ms
495:	total: 20.9s	remaining: 168ms
496:	total: 20.9s	remaining: 126ms
497:	total: 20.9s	remaining: 84.1ms
498:	total: 21s	remaining: 42.1ms
499:	total: 21s	remaining: 0us
1.0
Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:   16.7s
[Parallel(n_jobs=8)]: Done  75 out of  75 | elapsed:   33.9s finished


1.0
Fitting 5 folds for each of 15 candidates, totalling 75 fits
[CV] subsample=0.8, n_estimators=800, min_child_weight=10, max_depth=3, gamma=5, colsample_bytree=1.0 


...

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.9s remaining:    0.0s


[CV]  subsample=0.8, n_estimators=800, min_child_weight=10, max_depth=3, gamma=5, colsample_bytree=1.0, total=  12.9s
[CV] subsample=0.8, n_estimators=800, min_child_weight=10, max_depth=3, gamma=5, colsample_bytree=1.0 
[CV]  subsample=0.8, n_estimators=800, min_child_weight=10, max_depth=3, gamma=5, colsample_bytree=1.0, total=  12.6s
[CV] subsample=0.8, n_estimators=800, min_child_weight=10, max_depth=3, gamma=5, colsample_bytree=1.0 
[CV]  subsample=0.8, n_estimators=800, min_child_weight=10, max_depth=3, gamma=5, colsample_bytree=1.0, total=  12.6s
[CV] subsample=0.8, n_estimators=800, min_child_weight=10, max_depth=3, gamma=5, colsample_bytree=1.0 
[CV]  subsample=0.8, n_estimators=800, min_child_weight=10, max_depth=3, gamma=5, colsample_bytree=1.0, total=  12.7s
[CV] subsample=0.6, n_estimators=50, min_child_weight=10, max_depth=3, gamma=0.5, colsample_bytree=1.0 
[CV]  subsample=0.6, n_estimators=50, min_child_weight=10, max_depth=3, gamma=0.5, colsample_bytree=1.0, total=   1

[CV]  subsample=1.0, n_estimators=100, min_child_weight=10, max_depth=5, gamma=5, colsample_bytree=0.6, total=   2.9s
[CV] subsample=1.0, n_estimators=100, min_child_weight=10, max_depth=5, gamma=5, colsample_bytree=0.6 
[CV]  subsample=1.0, n_estimators=100, min_child_weight=10, max_depth=5, gamma=5, colsample_bytree=0.6, total=   2.9s
[CV] subsample=0.6, n_estimators=300, min_child_weight=1, max_depth=5, gamma=1.5, colsample_bytree=1.0 
[CV]  subsample=0.6, n_estimators=300, min_child_weight=1, max_depth=5, gamma=1.5, colsample_bytree=1.0, total=   7.3s
[CV] subsample=0.6, n_estimators=300, min_child_weight=1, max_depth=5, gamma=1.5, colsample_bytree=1.0 
[CV]  subsample=0.6, n_estimators=300, min_child_weight=1, max_depth=5, gamma=1.5, colsample_bytree=1.0, total=   7.3s
[CV] subsample=0.6, n_estimators=300, min_child_weight=1, max_depth=5, gamma=1.5, colsample_bytree=1.0 
[CV]  subsample=0.6, n_estimators=300, min_child_weight=1, max_depth=5, gamma=1.5, colsample_bytree=1.0, total=

[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:  8.0min finished


1.0


In [15]:
from sklearn.ensemble import VotingClassifier
estimators=[('xgb', txgb), ('lg', tlg), ('cat', tcat)]
#create our voting classifier, inputting our models
ensemble = VotingClassifier(estimators, voting='soft')
ensemble.fit(train, train.target.values)

0:	learn: 0.1334836	total: 58.4ms	remaining: 29.1s
1:	learn: 0.0270301	total: 112ms	remaining: 27.9s
2:	learn: 0.0068409	total: 162ms	remaining: 26.8s
3:	learn: 0.0021735	total: 205ms	remaining: 25.4s
4:	learn: 0.0008367	total: 251ms	remaining: 24.8s
5:	learn: 0.0003791	total: 304ms	remaining: 25s
6:	learn: 0.0001992	total: 350ms	remaining: 24.7s
7:	learn: 0.0001174	total: 392ms	remaining: 24.1s
8:	learn: 0.0000757	total: 433ms	remaining: 23.6s
9:	learn: 0.0000524	total: 481ms	remaining: 23.6s
10:	learn: 0.0000385	total: 524ms	remaining: 23.3s
11:	learn: 0.0000296	total: 566ms	remaining: 23s
12:	learn: 0.0000235	total: 608ms	remaining: 22.8s
13:	learn: 0.0000193	total: 649ms	remaining: 22.5s
14:	learn: 0.0000162	total: 690ms	remaining: 22.3s
15:	learn: 0.0000139	total: 735ms	remaining: 22.2s
16:	learn: 0.0000123	total: 777ms	remaining: 22.1s
17:	learn: 0.0000108	total: 816ms	remaining: 21.9s
18:	learn: 0.0000096	total: 865ms	remaining: 21.9s
19:	learn: 0.0000086	total: 912ms	remaining:

161:	learn: 0.0000045	total: 7s	remaining: 14.6s
162:	learn: 0.0000045	total: 7.04s	remaining: 14.6s
163:	learn: 0.0000045	total: 7.08s	remaining: 14.5s
164:	learn: 0.0000045	total: 7.12s	remaining: 14.5s
165:	learn: 0.0000045	total: 7.17s	remaining: 14.4s
166:	learn: 0.0000045	total: 7.22s	remaining: 14.4s
167:	learn: 0.0000045	total: 7.27s	remaining: 14.4s
168:	learn: 0.0000045	total: 7.3s	remaining: 14.3s
169:	learn: 0.0000045	total: 7.34s	remaining: 14.3s
170:	learn: 0.0000045	total: 7.38s	remaining: 14.2s
171:	learn: 0.0000045	total: 7.43s	remaining: 14.2s
172:	learn: 0.0000045	total: 7.48s	remaining: 14.1s
173:	learn: 0.0000045	total: 7.53s	remaining: 14.1s
174:	learn: 0.0000045	total: 7.56s	remaining: 14s
175:	learn: 0.0000045	total: 7.6s	remaining: 14s
176:	learn: 0.0000045	total: 7.64s	remaining: 14s
177:	learn: 0.0000045	total: 7.69s	remaining: 13.9s
178:	learn: 0.0000045	total: 7.74s	remaining: 13.9s
179:	learn: 0.0000045	total: 7.78s	remaining: 13.8s
180:	learn: 0.0000045	t

321:	learn: 0.0000045	total: 13.9s	remaining: 7.66s
322:	learn: 0.0000045	total: 13.9s	remaining: 7.62s
323:	learn: 0.0000045	total: 13.9s	remaining: 7.57s
324:	learn: 0.0000045	total: 14s	remaining: 7.53s
325:	learn: 0.0000045	total: 14s	remaining: 7.49s
326:	learn: 0.0000045	total: 14.1s	remaining: 7.44s
327:	learn: 0.0000045	total: 14.1s	remaining: 7.4s
328:	learn: 0.0000045	total: 14.2s	remaining: 7.36s
329:	learn: 0.0000045	total: 14.2s	remaining: 7.32s
330:	learn: 0.0000045	total: 14.3s	remaining: 7.28s
331:	learn: 0.0000045	total: 14.3s	remaining: 7.23s
332:	learn: 0.0000045	total: 14.3s	remaining: 7.19s
333:	learn: 0.0000045	total: 14.4s	remaining: 7.14s
334:	learn: 0.0000045	total: 14.4s	remaining: 7.1s
335:	learn: 0.0000045	total: 14.4s	remaining: 7.05s
336:	learn: 0.0000045	total: 14.5s	remaining: 7.01s
337:	learn: 0.0000045	total: 14.5s	remaining: 6.96s
338:	learn: 0.0000045	total: 14.6s	remaining: 6.92s
339:	learn: 0.0000045	total: 14.6s	remaining: 6.87s
340:	learn: 0.0000

482:	learn: 0.0000045	total: 20.6s	remaining: 726ms
483:	learn: 0.0000045	total: 20.7s	remaining: 683ms
484:	learn: 0.0000045	total: 20.7s	remaining: 640ms
485:	learn: 0.0000045	total: 20.7s	remaining: 598ms
486:	learn: 0.0000045	total: 20.8s	remaining: 555ms
487:	learn: 0.0000045	total: 20.8s	remaining: 512ms
488:	learn: 0.0000045	total: 20.9s	remaining: 469ms
489:	learn: 0.0000045	total: 20.9s	remaining: 427ms
490:	learn: 0.0000045	total: 20.9s	remaining: 384ms
491:	learn: 0.0000045	total: 21s	remaining: 341ms
492:	learn: 0.0000045	total: 21s	remaining: 299ms
493:	learn: 0.0000045	total: 21.1s	remaining: 256ms
494:	learn: 0.0000045	total: 21.1s	remaining: 213ms
495:	learn: 0.0000045	total: 21.2s	remaining: 171ms
496:	learn: 0.0000045	total: 21.2s	remaining: 128ms
497:	learn: 0.0000045	total: 21.2s	remaining: 85.3ms
498:	learn: 0.0000045	total: 21.3s	remaining: 42.6ms
499:	learn: 0.0000045	total: 21.3s	remaining: 0us


VotingClassifier(estimators=[('xgb',
                              XGBClassifier(base_score=0.5, booster='gbtree',
                                            colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=1.0, gamma=5,
                                            learning_rate=0.1, max_delta_step=0,
                                            max_depth=3, min_child_weight=10,
                                            missing=None, n_estimators=800,
                                            n_jobs=1, nthread=None,
                                            objective='binary:logistic',
                                            random_state=0, reg_alpha=0,
                                            reg_lambda=1, scale_pos_wei...
                                             min_child_weight=10000.0,
                                             min_split_gain=0.0,
                     

In [16]:
oof_preds2 = np.zeros((len(train)))
test_preds2 = np.zeros((len(test)))
cvs2 = []
skf = StratifiedKFold(n_splits=20)
for train_index, test_index in tqdm(skf.split(train, train.target.values)):
    X_train, X_test = train.iloc[train_index, :], train.iloc[test_index, :]
    y_train, y_test = X_train.target, X_test.target.values

    X_train.drop(['id','target'], inplace=True, axis=1)
    X_test.drop(['id','target'], inplace=True, axis=1)
    
    ensemble = VotingClassifier(estimators, voting='soft')
    ensemble.fit(X_train,y_train)
             
    valid_fold_preds = ensemble.predict(X_test)
    test_fold_preds = ensemble.predict(test.drop(['id','target'], axis=1))
    oof_preds2[test_index] = valid_fold_preds
    test_preds2 += test_fold_preds
    cvs2.append( metrics.roc_auc_score(y_test, valid_fold_preds))
    print( cvs2[-1])
   

0it [00:00, ?it/s]

0:	learn: 0.6523552	total: 47.3ms	remaining: 23.6s
1:	learn: 0.6258622	total: 86.4ms	remaining: 21.5s
2:	learn: 0.6093056	total: 125ms	remaining: 20.8s
3:	learn: 0.5969394	total: 164ms	remaining: 20.4s
4:	learn: 0.5883010	total: 203ms	remaining: 20.1s
5:	learn: 0.5811795	total: 243ms	remaining: 20s
6:	learn: 0.5763646	total: 292ms	remaining: 20.5s
7:	learn: 0.5720172	total: 339ms	remaining: 20.9s
8:	learn: 0.5681469	total: 390ms	remaining: 21.3s
9:	learn: 0.5650703	total: 429ms	remaining: 21s
10:	learn: 0.5623442	total: 472ms	remaining: 21s
11:	learn: 0.5601109	total: 518ms	remaining: 21.1s
12:	learn: 0.5577469	total: 563ms	remaining: 21.1s
13:	learn: 0.5557295	total: 608ms	remaining: 21.1s
14:	learn: 0.5539284	total: 648ms	remaining: 21s
15:	learn: 0.5522216	total: 685ms	remaining: 20.7s
16:	learn: 0.5508508	total: 723ms	remaining: 20.5s
17:	learn: 0.5491639	total: 763ms	remaining: 20.4s
18:	learn: 0.5478733	total: 805ms	remaining: 20.4s
19:	learn: 0.5465004	total: 852ms	remaining: 20

161:	learn: 0.5124261	total: 6.9s	remaining: 14.4s
162:	learn: 0.5123414	total: 6.95s	remaining: 14.4s
163:	learn: 0.5123203	total: 6.98s	remaining: 14.3s
164:	learn: 0.5122901	total: 7.02s	remaining: 14.3s
165:	learn: 0.5122584	total: 7.07s	remaining: 14.2s
166:	learn: 0.5122252	total: 7.11s	remaining: 14.2s
167:	learn: 0.5121732	total: 7.15s	remaining: 14.1s
168:	learn: 0.5121500	total: 7.18s	remaining: 14.1s
169:	learn: 0.5121206	total: 7.22s	remaining: 14s
170:	learn: 0.5120706	total: 7.27s	remaining: 14s
171:	learn: 0.5120359	total: 7.31s	remaining: 13.9s
172:	learn: 0.5120130	total: 7.35s	remaining: 13.9s
173:	learn: 0.5120012	total: 7.39s	remaining: 13.8s
174:	learn: 0.5119649	total: 7.43s	remaining: 13.8s
175:	learn: 0.5118640	total: 7.48s	remaining: 13.8s
176:	learn: 0.5118439	total: 7.52s	remaining: 13.7s
177:	learn: 0.5118121	total: 7.58s	remaining: 13.7s
178:	learn: 0.5117884	total: 7.62s	remaining: 13.7s
179:	learn: 0.5117567	total: 7.66s	remaining: 13.6s
180:	learn: 0.511

320:	learn: 0.5084127	total: 13.7s	remaining: 7.63s
321:	learn: 0.5083631	total: 13.7s	remaining: 7.59s
322:	learn: 0.5083502	total: 13.8s	remaining: 7.55s
323:	learn: 0.5083244	total: 13.8s	remaining: 7.51s
324:	learn: 0.5082919	total: 13.9s	remaining: 7.47s
325:	learn: 0.5082771	total: 13.9s	remaining: 7.43s
326:	learn: 0.5082708	total: 14s	remaining: 7.39s
327:	learn: 0.5082532	total: 14s	remaining: 7.35s
328:	learn: 0.5082459	total: 14.1s	remaining: 7.31s
329:	learn: 0.5081597	total: 14.1s	remaining: 7.27s
330:	learn: 0.5081487	total: 14.2s	remaining: 7.23s
331:	learn: 0.5081297	total: 14.2s	remaining: 7.19s
332:	learn: 0.5080977	total: 14.3s	remaining: 7.15s
333:	learn: 0.5080874	total: 14.3s	remaining: 7.11s
334:	learn: 0.5080622	total: 14.4s	remaining: 7.07s
335:	learn: 0.5080553	total: 14.4s	remaining: 7.03s
336:	learn: 0.5080450	total: 14.5s	remaining: 6.99s
337:	learn: 0.5080345	total: 14.5s	remaining: 6.95s
338:	learn: 0.5080259	total: 14.5s	remaining: 6.91s
339:	learn: 0.50

479:	learn: 0.5058468	total: 20.6s	remaining: 859ms
480:	learn: 0.5058264	total: 20.7s	remaining: 816ms
481:	learn: 0.5057869	total: 20.7s	remaining: 773ms
482:	learn: 0.5057624	total: 20.7s	remaining: 730ms
483:	learn: 0.5057543	total: 20.8s	remaining: 687ms
484:	learn: 0.5057378	total: 20.8s	remaining: 644ms
485:	learn: 0.5057312	total: 20.8s	remaining: 601ms
486:	learn: 0.5057271	total: 20.9s	remaining: 558ms
487:	learn: 0.5057177	total: 20.9s	remaining: 515ms
488:	learn: 0.5057068	total: 21s	remaining: 472ms
489:	learn: 0.5057000	total: 21s	remaining: 429ms
490:	learn: 0.5056928	total: 21s	remaining: 386ms
491:	learn: 0.5056840	total: 21.1s	remaining: 343ms
492:	learn: 0.5056483	total: 21.1s	remaining: 300ms
493:	learn: 0.5056408	total: 21.2s	remaining: 257ms
494:	learn: 0.5056371	total: 21.2s	remaining: 214ms
495:	learn: 0.5056082	total: 21.3s	remaining: 171ms
496:	learn: 0.5056001	total: 21.3s	remaining: 129ms
497:	learn: 0.5055926	total: 21.4s	remaining: 85.8ms
498:	learn: 0.505

...

164:	learn: 0.5121135	total: 7.01s	remaining: 14.2s
165:	learn: 0.5120969	total: 7.05s	remaining: 14.2s
166:	learn: 0.5120548	total: 7.1s	remaining: 14.2s
167:	learn: 0.5120299	total: 7.14s	remaining: 14.1s
168:	learn: 0.5120049	total: 7.17s	remaining: 14.1s
169:	learn: 0.5119821	total: 7.21s	remaining: 14s
170:	learn: 0.5119618	total: 7.25s	remaining: 14s
171:	learn: 0.5119329	total: 7.3s	remaining: 13.9s
172:	learn: 0.5119145	total: 7.34s	remaining: 13.9s
173:	learn: 0.5118180	total: 7.37s	remaining: 13.8s
174:	learn: 0.5118047	total: 7.42s	remaining: 13.8s
175:	learn: 0.5117829	total: 7.46s	remaining: 13.7s
176:	learn: 0.5117512	total: 7.5s	remaining: 13.7s
177:	learn: 0.5117382	total: 7.54s	remaining: 13.6s
178:	learn: 0.5116264	total: 7.6s	remaining: 13.6s
179:	learn: 0.5116048	total: 7.65s	remaining: 13.6s
180:	learn: 0.5115851	total: 7.7s	remaining: 13.6s
181:	learn: 0.5115125	total: 7.75s	remaining: 13.5s
182:	learn: 0.5114910	total: 7.8s	remaining: 13.5s
183:	learn: 0.5114430	

323:	learn: 0.5081129	total: 13.7s	remaining: 7.45s
324:	learn: 0.5081022	total: 13.8s	remaining: 7.41s
325:	learn: 0.5080927	total: 13.8s	remaining: 7.37s
326:	learn: 0.5080222	total: 13.8s	remaining: 7.32s
327:	learn: 0.5080119	total: 13.9s	remaining: 7.28s
328:	learn: 0.5080003	total: 13.9s	remaining: 7.24s
329:	learn: 0.5079755	total: 14s	remaining: 7.2s
330:	learn: 0.5079644	total: 14s	remaining: 7.16s
331:	learn: 0.5079535	total: 14.1s	remaining: 7.11s
332:	learn: 0.5079227	total: 14.1s	remaining: 7.07s
333:	learn: 0.5079097	total: 14.1s	remaining: 7.02s
334:	learn: 0.5078838	total: 14.2s	remaining: 6.98s
335:	learn: 0.5078770	total: 14.2s	remaining: 6.94s
336:	learn: 0.5078668	total: 14.3s	remaining: 6.89s
337:	learn: 0.5078545	total: 14.3s	remaining: 6.86s
338:	learn: 0.5078452	total: 14.4s	remaining: 6.82s
339:	learn: 0.5078323	total: 14.4s	remaining: 6.77s
340:	learn: 0.5077723	total: 14.4s	remaining: 6.73s
341:	learn: 0.5077659	total: 14.5s	remaining: 6.69s
342:	learn: 0.507

483:	learn: 0.5058297	total: 20.4s	remaining: 675ms
484:	learn: 0.5058248	total: 20.5s	remaining: 633ms
485:	learn: 0.5058201	total: 20.5s	remaining: 591ms
486:	learn: 0.5058043	total: 20.6s	remaining: 549ms
487:	learn: 0.5057551	total: 20.6s	remaining: 506ms
488:	learn: 0.5057456	total: 20.6s	remaining: 464ms
489:	learn: 0.5057341	total: 20.7s	remaining: 422ms
490:	learn: 0.5057280	total: 20.7s	remaining: 380ms
491:	learn: 0.5057083	total: 20.8s	remaining: 338ms
492:	learn: 0.5056901	total: 20.8s	remaining: 296ms
493:	learn: 0.5056830	total: 20.9s	remaining: 253ms
494:	learn: 0.5056749	total: 20.9s	remaining: 211ms
495:	learn: 0.5056658	total: 20.9s	remaining: 169ms
496:	learn: 0.5056604	total: 21s	remaining: 127ms
497:	learn: 0.5056502	total: 21s	remaining: 84.4ms
498:	learn: 0.5056450	total: 21.1s	remaining: 42.2ms
499:	learn: 0.5055971	total: 21.1s	remaining: 0us


20it [50:48, 151.22s/it]

0.6333480461322136


In [22]:
total = sum(cvs)/len(cvs)+sum(cvs2)/len(cvs)-1
nn_w = (sum(cvs)/len(cvs)-0.5)/total
ens_w = (sum(cvs2)/len(cvs2)-0.5)/total
nn_w, ens_w

(0.6850217878956167, 0.3149782121043832)

In [26]:
preds = nn_w * test_preds / 20 + ens_w * test_preds2 / 20

0.9480147059375357

In [28]:
sub = pd.read_csv('input/sample_submission.csv')
sub['target'] = preds
sub.to_csv('label_ens_nn.csv',index=False)